# Meddling around with gendered names

**NOTES:** 

fasttext-wiki and w2v-googlenews has both upper and lower case names

glove-twitter, glove-wiki are lowercased everywhere

conceptnet has no names

In [3]:
import gc
import gensim
import gensim.downloader as download_api
import numpy as np
import pandas as pd
import sklearn
from sklearn import cluster
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', 100)

df =  pd.read_csv("us-likelihood-of-gender-by-name-in-2014.csv")
df['nameLower'] = df.name.str.lower()
df.sex = df.sex.map({'F':0, 'M':1})
print("# names: ", len(df.name.unique()))

# names:  11316


In [4]:
# gensim models from https://github.com/RaRe-Technologies/gensim-data
model_names = [
    ['glove-twitter-50', "nameLower"],
    ['glove-twitter-200', "nameLower"],
    ['glove-wiki-gigaword-50', "nameLower"],
    ['glove-wiki-gigaword-300', "nameLower"],
    ['word2vec-google-news-300', "nameLower"],
    ['word2vec-google-news-300', "name"],
    ['fasttext-wiki-news-subwords-300', "nameLower"],
    ['fasttext-wiki-news-subwords-300', "name"],
]
for m in range(len(model_names)):
    model_name = model_names[m][0]
    model = download_api.load(model_name)
    name_col = model_names[m][1]
    nameList = df[name_col].unique()
    # pre allocate memory for efficiency
    dn = pd.DataFrame(np.zeros(
        (len(nameList),
         model.vector_size))) # Add names to the dataframe to keep track
    dn.index = nameList
    oovNames = [] # out of model vocab names
    for name in nameList:
        try:
            dn.loc[dn.index==name, :] = model[name]
        except KeyError:
            oovNames.append(name)
    # clean result to cluster properly
    dn = dn.loc[~dn.index.isin(oovNames)]
    # get clustering
    genderClusters = sklearn.cluster.k_means(dn, 2)[1]
    # align result to join with M/F labellings
    dn = pd.DataFrame(list(zip(genderClusters, dn.index)), 
                      columns=[model_name +"_"+ name_col, name_col])
    # Merge results into final table
    # outer join has NaN for oov names
    df = df.merge(dn, on=name_col, how="outer")
    # cleanup: never trust python GC with big objects
    model, dn = None, None
    gc.collect()
    print('"{0}"'.format(model_name), " OOV words:", len(oovNames))

"glove-twitter-50"  OOV words: 2573
"glove-twitter-200"  OOV words: 2573
"glove-wiki-gigaword-50"  OOV words: 3656
"glove-wiki-gigaword-300"  OOV words: 3656
"word2vec-google-news-300"  OOV words: 8795
"word2vec-google-news-300"  OOV words: 881
"fasttext-wiki-news-subwords-300"  OOV words: 7705
"fasttext-wiki-news-subwords-300"  OOV words: 2413


In [7]:
df.head()

,sex,name,gender.prob,nameLower,glove-twitter-50_nameLower,glove-twitter-200_nameLower,glove-wiki-gigaword-50_nameLower,glove-wiki-gigaword-300_nameLower,word2vec-google-news-300_nameLower,word2vec-google-news-300_name,fasttext-wiki-news-subwords-300_nameLower,fasttext-wiki-news-subwords-300_name
0,0,Elaine,1.0,elaine,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0,Cathy,1.0,cathy,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
2,0,Heidi,1.0,heidi,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
3,0,Vicki,1.0,vicki,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
4,0,Melinda,1.0,melinda,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0


In [6]:
df.corr()

,sex,gender.prob,glove-twitter-50_nameLower,glove-twitter-200_nameLower,glove-wiki-gigaword-50_nameLower,glove-wiki-gigaword-300_nameLower,word2vec-google-news-300_nameLower,word2vec-google-news-300_name,fasttext-wiki-news-subwords-300_nameLower,fasttext-wiki-news-subwords-300_name
sex,1.000000,-0.065590,-0.145386,0.164910,0.271286,0.274034,0.121566,-0.328894,0.138257,-0.097341
gender.prob,-0.065590,1.000000,0.098041,-0.100032,-0.099066,-0.103799,-0.022604,0.054302,-0.035980,0.054071
glove-twitter-50_nameLower,-0.145386,0.098041,1.000000,-0.926019,-0.537039,-0.558034,-0.323773,0.185854,-0.013408,0.053715
glove-twitter-200_nameLower,0.164910,-0.100032,-0.926019,1.000000,0.566617,0.590998,0.333085,-0.184194,0.056784,-0.082886
glove-wiki-gigaword-50_nameLower,0.271286,-0.099066,-0.537039,0.566617,1.000000,0.905164,0.262896,-0.192232,0.385791,-0.293149
glove-wiki-gigaword-300_nameLower,0.274034,-0.103799,-0.558034,0.590998,0.905164,1.000000,0.274762,-0.196869,0.352819,-0.275379
word2vec-google-news-300_nameLower,0.121566,-0.022604,-0.323773,0.333085,0.262896,0.274762,1.000000,0.173640,-0.027425,-0.123003
word2vec-google-news-300_name,-0.328894,0.054302,0.185854,-0.184194,-0.192232,-0.196869,0.173640,1.000000,0.068648,-0.266324
fasttext-wiki-news-subwords-300_nameLower,0.138257,-0.035980,-0.013408,0.056784,0.385791,0.352819,-0.027425,0.068648,1.000000,-0.597834
fasttext-wiki-news-subwords-300_name,-0.097341,0.054071,0.053715,-0.082886,-0.293149,-0.275379,-0.123003,-0.266324,-0.597834,1.000000


In [ ]:
from sklearn import cluster, metrics

n_clusters = test_set.CAMPAIGNID.nunique()

test_set['kms'] = cluster.KMeans(n_clusters=n_clusters
                        ).fit(SE).labels_
test_set['acs'] = cluster.AgglomerativeClustering(n_clusters=n_clusters, 
                                                  affinity='euclidean', linkage='ward'
                        ).fit(SE).labels_
# NOTE: spectral requres hyperparameter fiddling to get good performance
test_set['spc'] = cluster.SpectralClustering(n_clusters=n_clusters,
                                             affinity= 'poly',# 'nearest_neighbors',
                                             n_init=50, 
                                             degree=3, gamma=0.3,
                                             eigen_solver='arpack', # optional (faster)
                        ).fit(SE).labels_


print("MI   score: {0} | {1} | {2}".format(
    int(metrics.adjusted_mutual_info_score(test_set['kms'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_mutual_info_score(test_set['acs'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_mutual_info_score(test_set['spc'], test_set['CAMPAIGNID']) * 100),
))
print("RAND score: {0} | {1} | {2}".format(
    int(metrics.adjusted_rand_score(test_set['kms'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_rand_score(test_set['acs'], test_set['CAMPAIGNID']) * 100),
    int(metrics.adjusted_rand_score(test_set['spc'], test_set['CAMPAIGNID']) * 100),
))
print("F-M score:  {0} | {1} | {2}".format(
    int(metrics.fowlkes_mallows_score(test_set['kms'], test_set['CAMPAIGNID']) * 100),
    int(metrics.fowlkes_mallows_score(test_set['acs'], test_set['CAMPAIGNID']) * 100),
    int(metrics.fowlkes_mallows_score(test_set['spc'], test_set['CAMPAIGNID']) * 100),
))